# Homework 5

Alles Rebel
Computational Science PhD



## Intoduction!



## Pre-Reqs

Like other homeworks, we'll start by getting the kernel set up with the right dependencies. Running this once should do the trick for the rest of the homework;

In [1]:
import Pkg; 
Pkg.add("Distributions")
Pkg.add("PrettyTables")
Pkg.add("Plots")
Pkg.add("ForwardDiff")
Pkg.add("Optim")

using Printf
using LinearAlgebra
using ForwardDiff
using Random
using Distributions
using Statistics
using Dates
using PrettyTables
using Optim
using Plots

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
Precompiling project...
   3163.9 ms  ✓ Plots → IJuliaExt
  1 dependency successfully precompiled in 5 seconds. 240 already precompiled.
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


## Problem 1

The goal of this assignment is to apply a gradient solver to a constrainted optimization problem. In this case the problem needs to be atleast 10 dimensions. The method I'll stick with is the first one suggested by the assignment: the method of Augmented Lagrangian! And we'll apply it to Booth's Function, just updated to use 10 variables instead of 2 via the chaining method introduced in HW2, 1C.

### Broad Overview of Augmented Lagrangians
In general, these methods solve optimization problems with constraints by incorporating the constraints as additional terms in the objective function. These additional terms involve new parameters, known as Lagrange multipliers, which help reformulate the constrained problem (involving equalities or inequalities) into an unconstrained one.

To ensure numerical stablity - Augmented lagrangians intoduce a penality term to the discourage constraint violations. The optimization alternates between minimizing the augmented Lagrangian and updating the multipliers and penalty parameters iteratively. That's what we'll be implementing below!

### Booth's Function!

We'll reuse the code I wrote from the earlier homework - line for line. Originally designed to be used with existing autodiff (forwardiff in julia) and Optim package. It will suit our purposes for this homework. 

In [10]:
# Booth's function - generalized for any number of variables
# Uses the method suggested by Homework2 Problem 1C to chain variables
# pair wise to extend from 2D -> ND
# I wrote this for HW2
function booths_n(x; n = 10)
    result = 0.0
    # same rationale for ending at n-1, we need the nth element for n-1
    for i in 1:(n - 1)
        result += ((x[i] + 2*x[i+1] - 7)^2 + (2*x[i] + x[i+1] - 5)^2)
    end
    return result
end

# Same with this - since the methods of HW needed gradients
# we'll need this as well for this HW
function booths_n_gradient(x; n = length(x))
    grad = zeros(length(x)) # modified to be typeless
    
    # Special case for the first element
    grad[1] = 2*(x[1] + 2*x[2] - 7) + 4*(2*x[1] + x[2] - 5)
    
    # Compute the gradient for the 2:n-1 elements
    for i in 2:(n-1)
        grad[i] = 2*(x[i]+2*x[i+1]-7) + 4*(2*x[i] + x[i+1] - 5) + 4*(x[i-1] + 2*x[i] - 7) + 2*(2*x[i-1] + x[i] - 5)
    end

    # Special case for the last element (n-th element)
    grad[n] = 4*(x[n-1] + 2*x[n] - 7) + 2*(2*x[n-1] + x[n] - 5) 
    
    return grad
end

# since this is a constraints optimization problem, we'll add something here

# Define the constraint h(x)
function h(x)
    return [sum(x)]  # For example, we can constrain the sum of x to zero
end    

h (generic function with 1 method)

### Implemention of augmented lagrangian method

This will be a bit involved, but in general - it's the following steps:
- We'll attempt to use Julia's Optim library for the minimization method
- We'll instrument up the result to match the results from earlier
(function calls, grad calls, convergement measure, optimization point)

In [8]:
# Augmented Lagrangian Method using Optim.optimize
# Based upon the code in Algorithms for Optimization (Algorithm 10.2 of K&W)
# Modified to keep track of function and grad calls
function augmented_lagrange_method(f, h, x0, k_max; ρ=1.0, γ=2.0, tol=1e-6)
    x = x0
    λ = zeros(length(h(x)))               # Initialize Lagrange multipliers
    old_f_value = f(x)                    # Track old objective value
    convergence_measure = 0.0             # Convergence measure

    total_f_calls = 0   # Total function evaluations
    total_g_calls = 0   # Total gradient evaluations

    for k in 1:k_max
        # Define penalty function
        function p(x)
            λ_promoted = convert.(eltype(x[1]), λ)
            return (ρ / 2) * sum(h(x).^2) - dot(λ_promoted, h(x))
        end

        # Define augmented objective function
        function augmented_objective(x)
            return f(x) + p(x)
        end

        # Optimize using Optim.optimize
        result = optimize(augmented_objective, x, method=LBFGS(); autodiff=:forward)
        x = Optim.minimizer(result)

        # Accumulate function and gradient evaluations
        total_f_calls += result.f_calls
        total_g_calls += result.g_calls

        # Update convergence metrics
        new_f_value = f(x)
        convergence_measure = abs(new_f_value - old_f_value)
        old_f_value = new_f_value

        # Check for convergence
        if convergence_measure < tol
            break
        end

        # Update Lagrange multipliers and penalty parameter
        λ .= λ .- ρ * ForwardDiff.value.(h(x))
        ρ *= γ
    end

    return x, total_f_calls, total_g_calls
end

augmented_lagrange_method (generic function with 1 method)

### Experimental Methodology

We'll utilize the same testing methodology as the HW2, generating a BUNCH of random vectors to start from. And then running the method across each of the vectors, recording how many function calls, gradient calls, exectuation time and error from what the ideal solution would have been. We'll compare the above method against the earlier line search method.

First we'll get the environment ready to do the experiment - how we'll generate everything an calculate the answers. 

In [9]:
# The code we used earlier in HW2! This will be what we'll compare against
# I wrote this code with the book as the primary sources
# Conjugate Gradient Descent method (with function and gradient counts)
# Updated tracking - and fixed accounting issues missed in originial HW2 implementation
mutable struct ConjugateGradientDescent
    d::Vector{Float64}
    g::Vector{Float64}
    iterations::Int
    func_calls::Int
    grad_calls::Int
    converged::Bool
end

function init!(M::ConjugateGradientDescent, f, ∇f, x)
    M.g = ∇f(x)
    M.d = -M.g
    M.iterations = 0
    M.func_calls = 1
    M.grad_calls = 1
    M.converged = false
    return M
end

function backtracking_line_search(f, ∇f, x, d, α; p=0.5, β=1e-4)
    y, g = f(x), ∇f(x)
    func_calls = 1
    grad_calls = 1
    while f(x + α*d) > y + β*α*(g⋅d)
        α *= p
        func_calls += 1
    end
    return α, func_calls, grad_calls
end

function step!(M::ConjugateGradientDescent, f, ∇f, x)
    d, g = M.d, M.g
    α, f_calls_ls, g_calls_ls = backtracking_line_search(f, ∇f, x, d, 1.0)  # initial α can be 1.0
    x_new = x + α * d
    M.func_calls += f_calls_ls
    M.grad_calls += g_calls_ls
    M.func_calls += 1  # For evaluating f at x_new
    g_new = ∇f(x_new)
    M.grad_calls += 1
    β = max(0, dot(g_new, g_new - g) / dot(g, g))
    d_new = -g_new + β * d
    M.d = d_new
    M.g = g_new
    M.iterations += 1
    if norm(g_new) < 1e-6
        M.converged = true
    end
    return x_new
end

step! (generic function with 1 method)

In [12]:
function generate_initial_points(left_limit=-10.0, right_limit=10.0, samples=100, dims=10)
    # Generate 100 random real-valued starting points as specified
    initial_points = [] 

    for i in 1:samples # defaults to 100 samples
        this_sample = []
        
        for j in 1:dims
            # y is uniformly picked in [-10.0, 10.0] by default
            # note, rand is already uniform, just need to cap the limits
            y = rand(Uniform(left_limit, right_limit))
            # choose + or - with equal probability, we'll use bitrand (50/50)
            sign = bitrand() == 0 ? -1 : 1
            x = sign * exp(y)
            push!(this_sample, x)
        end
        
        push!(initial_points, this_sample)
    end

    return initial_points
end


generate_initial_points (generic function with 5 methods)

In [11]:
# Generate initial points for Booth's function
initial_points_booths = generate_initial_points()  # Using multiple samples

# Get the analytical solution using LBFGS (for comparison)
booths_opt_res = optimize(booths_n, convert(Vector{Float64}, initial_points_booths[1]), LBFGS(); autodiff=:forward)
booths_opt_min_x = Optim.minimizer(booths_opt_res)

# Lists to store results
results_conj_booths = []
results_aug_lag_booths = []

# Run the optimizer for each method
for (method_name, results) in zip(["Conjugate Gradient Descent", "Augmented Lagrangian Method"], [results_conj_booths, results_aug_lag_booths])
    for x0 in initial_points_booths
        x0 = convert(Vector{Float64}, x0)
        start_time = time()
        if method_name == "Conjugate Gradient Descent"
            # Initialize CGD
            M = ConjugateGradientDescent(zeros(length(x0)), zeros(length(x0)), 0, 0, 0, false)
            init!(M, booths_n, booths_n_gradient, x0)
            iteration_limit = 1000
            x = x0
            for i in 1:iteration_limit
                x = step!(M, booths_n, booths_n_gradient, x)
                if M.converged
                    break
                end
                if any(isnan, x) || any(isinf, x)
                    break
                end
            end
            elapsed_time = time() - start_time
            f_calls, g_calls, iterations, converged = M.func_calls, M.grad_calls, M.iterations, M.converged
            absolute_error = norm(x - booths_opt_min_x, Inf)
            push!(results, (absolute_error, converged, f_calls, g_calls, elapsed_time))
        elseif method_name == "Augmented Lagrangian Method"
            # Use augmented Lagrangian method
            k_max = 10  # Maximum number of outer iterations
            x_opt, total_f_calls, total_g_calls = augmented_lagrange_method(booths_n, h, x0, k_max)
            elapsed_time = time() - start_time
            absolute_error = norm(x_opt - booths_opt_min_x, Inf)
            push!(results, (absolute_error, total_f_calls, total_g_calls, elapsed_time))
        end
    end
end

# Print results for Conjugate Gradient Descent
println("Results for Conjugate Gradient Descent on Booth's function:")
for (i, res) in enumerate(results_conj_booths)
    println("Run $i: Absolute Error = $(res[1]), Converged = $(res[2]), Function Calls = $(res[3]), Gradient Calls = $(res[4]), Time = $(res[5])")
end

# Print results for Augmented Lagrangian Method
println("\nResults for Augmented Lagrangian Method on Booth's function:")
for (i, res) in enumerate(results_aug_lag_booths)
    println("Run $i: Absolute Error = $(res[1]), Total Function Calls = $(res[2]), Total Gradient Calls = $(res[3]), Time = $(res[4])")
end


Results for Conjugate Gradient Descent on Booth's function:
Run 1: Absolute Error = 2.2235711272244885e-8, Converged = true, Function Calls = 2307, Gradient Calls = 225, Time = 0.17394399642944336
Run 2: Absolute Error = 4.117869822906073e-8, Converged = true, Function Calls = 1337, Gradient Calls = 165, Time = 0.0002181529998779297
Run 3: Absolute Error = 3.744888488199649e-8, Converged = true, Function Calls = 1374, Gradient Calls = 161, Time = 0.00020194053649902344
Run 4: Absolute Error = 1.64623950027476e-8, Converged = true, Function Calls = 1442, Gradient Calls = 179, Time = 0.0002110004425048828
Run 5: Absolute Error = 2.902422346906519e-8, Converged = true, Function Calls = 1303, Gradient Calls = 183, Time = 0.0002219676971435547
Run 6: Absolute Error = 8.629066350351877e-9, Converged = true, Function Calls = 1271, Gradient Calls = 157, Time = 0.00018310546875
Run 7: Absolute Error = 3.844521079265917e-8, Converged = true, Function Calls = 1562, Gradient Calls = 161, Time = 0.

Run 75: Absolute Error = 2.085054884304327e-8, Converged = true, Function Calls = 1775, Gradient Calls = 183, Time = 0.0002319812774658203
Run 76: Absolute Error = 5.895660315502482e-8, Converged = true, Function Calls = 1233, Gradient Calls = 155, Time = 0.00017213821411132812
Run 77: Absolute Error = 3.002349613367983e-8, Converged = true, Function Calls = 1457, Gradient Calls = 185, Time = 0.00020599365234375
Run 78: Absolute Error = 4.389560448458951e-8, Converged = true, Function Calls = 1312, Gradient Calls = 163, Time = 0.00018787384033203125
Run 79: Absolute Error = 3.880525767385734e-8, Converged = true, Function Calls = 972, Gradient Calls = 129, Time = 0.00014209747314453125
Run 80: Absolute Error = 7.649691502464862e-8, Converged = true, Function Calls = 1987, Gradient Calls = 209, Time = 0.00025916099548339844
Run 81: Absolute Error = 7.189246553629403e-8, Converged = true, Function Calls = 1494, Gradient Calls = 183, Time = 0.00020694732666015625
Run 82: Absolute Error = 

Run 52: Absolute Error = 5.702341137037654, Total Function Calls = 181, Total Gradient Calls = 181, Time = 0.0003170967102050781
Run 53: Absolute Error = 5.70234113609602, Total Function Calls = 174, Total Gradient Calls = 174, Time = 0.00031113624572753906
Run 54: Absolute Error = 5.702341137037654, Total Function Calls = 179, Total Gradient Calls = 179, Time = 0.0003190040588378906
Run 55: Absolute Error = 5.702341137037655, Total Function Calls = 181, Total Gradient Calls = 181, Time = 0.0003180503845214844
Run 56: Absolute Error = 5.702341137037654, Total Function Calls = 178, Total Gradient Calls = 178, Time = 0.00031685829162597656
Run 57: Absolute Error = 5.702341136096021, Total Function Calls = 174, Total Gradient Calls = 174, Time = 0.00031495094299316406
Run 58: Absolute Error = 5.702341136096018, Total Function Calls = 173, Total Gradient Calls = 173, Time = 0.00030517578125
Run 59: Absolute Error = 5.702341136096012, Total Function Calls = 175, Total Gradient Calls = 175, 